In [1]:
%load_ext sql
#%sql --section duck
#import epyfun
#sqlite_file = epyfun.get_latest_file("./data/")
%sql duckdb:///:memory:
%sql ATTACH './duckdb/strom.duckdb' (READ_ONLY);
%sql SHOW DATABASES;
%sql USE strom;

Loading configurations from C:\OneDrive\All\strom\pyproject.toml.

Settings changed:

Config,value
feedback,True
autopandas,True
displaycon,False
dsn_filename,./connections.ini


Connecting to 'duckdb:///:memory:'

,Success


In [4]:
%%sql


SELECT * FROM strom_minute WHERE cm IS NULL OR cm = 'NaN' OR cm = 'Infinity' OR cm = '-Infinity';

SELECT * FROM strom_minute WHERE minute > '2024-05-08 08:09:00' AND minute < '2024-05-08 09:11:00';

SELECT * FROM strom WHERE date > '2024-05-07 08:09:00' AND date < '2024-05-09 08:09:00' ORDER BY meterid, date;

SELECT SUM(cm) from strom_minute;

,sum(cm)
0,63373.324935


In [2]:
%%sql



SELECT     
    MIN(value) AS Min,
    MAX(value) AS Max,
    MAX(value) - MIN(value) AS Use2,
    SUM(cm) AS Use,
    MIN(date) AS First,
    MAX(date) AS Last,
    date_sub('minute', MIN(minute), MAX(minute)) AS Mins, 
    24.0 * 60.0 * Use / Mins AS "Use/Day",
    365.25 * "Use/Day" AS "Use/Year",
    0.3713 * "Use/Day" AS "Daily Exp",
    0.3713 * "Use/Year" AS "Yearly Exp"
FROM strom_minute
WHERE minute >= '2016-07-04 07:50:00' AND minute <= '2025-02-01 16:26:00' AND
meterid IN (1)
;


,Min,Max,Use2,Use,First,Last,Mins,Use/Day,Use/Year,Daily Exp,Yearly Exp
0,0,23128,23128,NaN,2016-07-04 07:50:00,2025-02-01 16:26:00,4513476,NaN,NaN,NaN,NaN


In [4]:
%%sql

  WITH minutes_table AS (
      SELECT 
          UNNEST(generate_series(ts[1], ts[2], interval 1 minute)) 
              AS minute, 
          generate_series AS meterid
      FROM (VALUES ([(
          SELECT MIN(date) FROM strom), (SELECT MAX(date) FROM strom)]
      )) t(ts), generate_series(1, 3)
  )
  SELECT 
      minutes_table.meterid,
      minutes_table.minute,
      FIRST_VALUE(strom.cm IGNORE NULLS) OVER(
          PARTITION BY minutes_table.meterid 
          ORDER BY minutes_table.minute 
          ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING 
      ) AS cm,
      strom.date,
      strom.value,
      strom.minutes,
      strom.consumption 
  FROM minutes_table
  LEFT JOIN strom
  ON minutes_table.minute = strom.date AND 
      minutes_table.meterid = strom.meterid
  ORDER BY minutes_table.minute, minutes_table.meterid
  ;


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,meterid,minute,cm,date,value,minutes,consumption
0,1,2016-07-04 07:50:00,0.100000,2016-07-04 07:50:00,0.0,NaN,NaN
1,2,2016-07-04 07:50:00,0.100000,2016-07-04 07:50:00,0.0,NaN,NaN
2,3,2016-07-04 07:50:00,0.100000,2016-07-04 07:50:00,0.0,NaN,NaN
3,1,2016-07-04 07:51:00,0.100000,NaT,NaN,NaN,NaN
4,2,2016-07-04 07:51:00,0.100000,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...
12644263,2,2024-07-09 05:44:00,0.001733,NaT,NaN,NaN,NaN
12644264,3,2024-07-09 05:44:00,0.000000,NaT,NaN,NaN,NaN
12644265,1,2024-07-09 05:45:00,0.003472,2024-07-09 05:45:00,496.0,576.0,2.0
12644266,2,2024-07-09 05:45:00,0.001733,2024-07-09 05:45:00,198.0,577.0,1.0


,year,month,day,hour,nd,wd,nt,ht,weight
0,2020,12,1,1,0.273349,1.231004,0.820046,0.410959,5.196749e-06
1,2020,12,1,21,0.545455,1.367372,0.821918,0.545455,2.072410e-05
2,2020,12,2,1,0.000000,0.669643,0.401786,0.267857,4.989895e-06
3,2020,12,2,21,0.289157,1.084337,0.289157,0.795181,1.451589e-06
4,2020,12,3,4,0.160000,1.120000,0.480000,0.640000,7.111111e-06
...,...,...,...,...,...,...,...,...,...
17550,2023,9,16,17,0.407793,0.170885,0.112629,0.058256,4.189858e-09
17551,2023,9,17,2,0.407793,0.170885,0.112629,0.058256,4.189858e-09
17552,2023,9,17,10,0.407793,0.170885,0.112629,0.058256,4.189858e-09
17553,2023,9,17,18,0.407793,0.170885,0.112629,0.058256,4.189858e-09


In [1]:
%%sql
SELECT *,
            1 / FIRST_VALUE(minutes IGNORE NULLS) OVER(
                PARTITION BY meterid 
                ORDER BY minute 
                ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING 
            ) AS weight,

FROM strom_minute
WHERE meterid = 1
--LIMIT 50 OFFSET 100089
;

UsageError: Cell magic `%%sql` not found.


In [3]:
%load_ext sql
%sql duckdb://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [6]:
%%sql

CREATE TABLE services AS
    FROM 'https://blobs.duckdb.org/nl-railway/services-2023.csv.gz';

RuntimeError: (duckdb.duckdb.CatalogException) Catalog Error: Table with name "services" already exists!
[SQL: CREATE TABLE services AS
    FROM 'https://blobs.duckdb.org/nl-railway/services-2023.csv.gz';]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [8]:
%%sql

SELECT format('{:,}', count(*)) AS num_services
FROM services;

,num_services
0,"21,239,393"


In [13]:
%%sql

DESCRIBE TABLE services;

,Success


In [1]:
import scipy.optimize as opt


# Define the function for the present value of an annuity
def present_value_annuity(r, X, PMT, n):
    return PMT * (1 - (1 + r) ** -n) / r - X


# Initial values
X = 37840  # Example initial amount
PMT = 242.57
n = 300

# Use a solver to find the interest rate
r = opt.newton(present_value_annuity, x0=0.05 / 12, args=(X, PMT, n))
annual_interest_rate = r * 12 * 100  # Convert monthly rate to annual percentage rate

print(f"The annual interest rate is approximately {annual_interest_rate:.7f}%")

The annual interest rate is approximately 5.9466018%


In [4]:
def calculate_monthly_payment(X, N, i):
    # Convert annual interest rate to monthly interest rate
    r = i / 12

    # Calculate the monthly payment using the formula
    PMT = X * (r * (1 + r) ** N) / ((1 + r) ** N - 1)

    return PMT


# Example values
X = 37840  # Loan amount
N = 15 * 12  # Total periods (months)
i = 0.04  # Annual interest rate (5%)

# Calculate monthly payment
monthly_payment = calculate_monthly_payment(X, N, i)
print(f"The monthly payment is approximately ${monthly_payment:.2f}")

The monthly payment is approximately $279.90


In [5]:
# | eval: false
# | include: false
import pandas as pd
import numpy as np
import seaborn as sns
import epyfun

import strom

strom_climate = strom.read_result("merge_strom_climate_data")
strom_climate["ds"] = strom_climate["date"].dt.date
strom_climate = strom_climate.set_index("ds")

cutoff = strom_climate["date"].max() - pd.DateOffset(years=1)

train_set = strom_climate[strom_climate["date"] <= cutoff]
test_set = strom_climate[strom_climate["date"] > cutoff]

X = strom_climate.drop(columns="wd")
y = strom_climate["wd"]

X_train = train_set.drop(columns="wd")
y_train = train_set["wd"]
X_test = test_set.drop(columns="wd")
y_test = test_set["wd"]

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

from sklego.preprocessing import ColumnSelector

pipe = Pipeline(
    [
        ("vars", ColumnSelector(columns=["tt_tu_mean", "rf_tu_mean"])),
        ("polynomial", PolynomialFeatures(degree=4)),
        ("model", LinearRegression()),
    ]
)

In [12]:
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(
    n_splits=5,
    # gap=7,
    max_train_size=None,
    test_size=15,
)

[{"train": len(split[0]), "test": len(split[1])} for split in list(tscv.split(X_train))]

[{'train': 344, 'test': 15},
 {'train': 359, 'test': 15},
 {'train': 374, 'test': 15},
 {'train': 389, 'test': 15},
 {'train': 404, 'test': 15}]

In [3]:
import plotly.graph_objects as go
import pandas as pd
from plotly import data

df = data.stocks()

layout = dict(
    hoversubplots="axis",
    title="Stock Price Changes",
    hovermode="x",
    grid=dict(rows=3, columns=1),
)

data = [
    go.Scatter(x=df["date"], y=df["AAPL"], xaxis="x", yaxis="y", name="Apple"),
    go.Scatter(x=df["date"], y=df["GOOG"], xaxis="x", yaxis="y2", name="Google"),
    go.Scatter(x=df["date"], y=df["AMZN"], xaxis="x", yaxis="y3", name="Amazon"),
]

fig = go.Figure(data=data, layout=layout)

fig.show()

In [ ]:
import plotly.graph_objects as go
import pandas as pd
from plotly import data

df = data.stocks()

layout = dict(
    hoversubplots="axis",
    title="Stock Price Changes",
    hovermode="x",
    grid=dict(rows=3, columns=1),
)

data = [
    go.Scatter(x=df["date"], y=df["AAPL"], xaxis="x", yaxis="y", name="Apple"),
    go.Scatter(x=df["date"], y=df["GOOG"], xaxis="x", yaxis="y2", name="Google"),
    go.Scatter(x=df["date"], y=df["AMZN"], xaxis="x", yaxis="y3", name="Amazon"),
]

fig = go.Figure(data=data, layout=layout)

fig.show()

In [34]:
import plotly.express as px
import pandas as pd
import epyfun

df = px.data.tips()
fig = px.violin(
    df,
    y="tip",
    x="smoker",
    color="sex",
    box=False,
    points=False,
    # hover_data=df.columns,
)
fig = fig.for_each_trace(
    lambda t: t.update(hovertemplate=epyfun.make_hover_template(df))
)
fig = fig.for_each_trace(lambda t: t.update(customdata=df))

fig = fig.update_traces(
    hovertemplate=epyfun.make_hover_template(df),
    customdata=pd.DataFrame(),
    # hoverlabel=["sdf", "sdf", "sdf", "sdf"]
)

fig = fig.update_layout(
    # xaxis=dict(visible=False),
    # https://plotly.com/python/reference/layout/xaxis/
    xaxis={
        "tickmode": "array",
        "tickvals": list(["a", "b"]),
        # "labelalias": {
        #     s: str(s)
        #     .replace("{", "")
        #     .replace("}", "")
        #     .replace("__", " ")
        #     .replace(", '", "<br>")
        #     .replace("'", "")
        #     for s in list(ordered_params)
        # },
        # "ticklabeloverflow": "allow",  # "allow" | "hide past div" | hide past domain
        # "ticktext": [str(s)[:3] for s in list(ordered_params)],
    },
)

fig.update_layout(hovermode=False)
fig.data
fig.show()